In [1]:
import os, sys
import gc
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append('../')
import torch

In [3]:
from get_algos import get_all_algos , run_experiment ,call_paths, plot_mean

In [4]:
import gym
# from gym.wrappers import Monitor
from stable_baselines3 import PPO, A2C, DDPG, DQN, SAC, TD3
from stable_baselines3.common.vec_env import VecFrameStack,VecVideoRecorder
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.monitor import Monitor

In [5]:
env_name='Breakout-v0'

### 2. Testing
<ol>
    <li>Create the environment</li>
    <li>For each algorithm:</li>
    <ol><li>Load the model</li>
        <li>Evaluate the model for 5 sample iterations </li>
        <li>Output the score  for each algo and each iteration</li>
        <li>Save renders of the output </li>
    </ol>
</ol> 

In [6]:
algo_list=get_all_algos(gym.make(env_name))
print(algo_list)

['PPO', 'A2C', 'DQN']


### 3. Testing

In [13]:
def testing_model(algo_list, env_name,n_steps,vid_length):
    total_rewards={}
    for algo_name in reversed(algo_list):
        algo_rewards=[]
        device='cpu' if algo_name=='DQN' else 'cuda'
        log_path, render_path, model_path=call_paths(algo_name,env_name,n_steps)
        env=VecVideoRecorder(VecFrameStack(make_atari_env(env_name,monitor_dir=render_path),n_stack=6),render_path,record_video_trigger=lambda step: step == 0,video_length=vid_length)
        model_name=env_name+"_"+algo_name+"_model"
        model=eval(algo_name).load(os.path.join(model_path, model_name),env,device=device)
        for i in range(5):
            state=env.reset()
            epi_rewards=0
            while True:
                action,_=model.predict(state)
                state,reward,done,_=env.step(action)
                epi_rewards+=reward[0]
                env.render()
                if done:
                    algo_rewards.append(epi_rewards)
                    break
        total_rewards[algo_name]=algo_rewards
        del model
        env.close()
    return total_rewards

In [14]:
test=total_rewards=testing_model(algo_list, env_name,n_steps=200000,vid_length=1000)

Wrapping the env in a VecTransposeImage.


E:\Anaconda\envs\ai_gym\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 84.69GB > 11.00GB
  "This system does not have apparently enough memory to store the complete "


Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.


In [15]:
test

{'DQN': [0.0, 0.0, 0.0, 5.0, 0.0],
 'A2C': [4.0, 0.0, 0.0, 0.0, 0.0],
 'PPO': [3.0, 0.0, 0.0, 3.0, 0.0]}

We see that PPO seems to consistently provide rewards with the highest average reward. Hence, we will use PPO as the algorithm for subsequent training of <b> 5,000,000 </b>steps